In [2]:
#Follow along exercises with http://neuralnetworksanddeeplearning.com/chap1.html
print("yo")

yo


Does this render in github?


Notes:
Perceptrons sufficient for universal computation because the can implement NAND. Kinda cool way to introduce them

$$w \cdot x = \sum w_j x_j$$

Exercise: NAND gate for addition
TODO: hack latex drawing here
x1 x2  a == NAND(x1,x2) b == NAND(x1,a) c== NAND(x2,a) low_bit == NAND(b,c) carry_bit== NAND(a,a)  result   base10
0  0        1            1                  1               0                   0                   00          0
0  1        1            1                  0               1                   0                   01          1
1  1        0            1                  1               0                   1                   10          2

Since Perceptrons can implment NAND, the can do this too


Exercise: Sigmoid neurons simulating perceptrons, part I 
Suppose we take all the weights and biases in a network of perceptrons, and multiply them by a positive constant, c>0. Show that the behaviour of the network doesn't change.

The activation are invariant under constant rescaling since you can write
$$w \cdot x > b $$

and $$w \cdot x >b \implies K(w \cdot x) > Kb$$

Exercise: Sigmoid neurons simulating perceptrons, part II 
Suppose we have the same setup as the last problem - a network of perceptrons. Suppose also that the overall input to the network of perceptrons has been chosen. We won't need the actual input value, we just need the input to have been fixed. Suppose the weights and biases are such that w⋅x+b≠0 for the input x to any particular perceptron in the network. Now replace all the perceptrons in the network by sigmoid neurons, and multiply the weights and biases by a positive constant c>0. Show that in the limit as c→∞ the behaviour of this network of sigmoid neurons is exactly the same as the network of perceptrons. How can this fail when w⋅x+b=0 for one of the perceptrons?

Well, sigmoid neurons is given by
$$\sigma(z) = \frac{1}{1+\exp^{-z}} $$
$$ \lim_{z->\infty} \sigma \rightarrow 0$$
$$ \lim_{z->-\infty} \sigma \rightarrow 1$$
and if $z = c w\cdot x$ then the sigmoid limit become undefined as $|c| \rightarrow \infty$ and $w \cdot x \rightarrow 0$


Exercis: There is a way of determining the bitwise representation of a digit by adding an extra layer to the three-layer network above. The extra layer converts the output from the previous layer into a binary representation, as illustrated in the figure below. Find a set of weights and biases for the new output layer. Assume that the first 3 layers of neurons are such that the correct output in the third layer (i.e., the old output layer) has activation at least 0.99, and incorrect outputs have activation less than 0.01.

In summary the 4 outputs represent the 4 bits that span the space of 0-15 decimal integers. You need the 

first bit to be active if the output is odd and inactive if even. 
second bit: if greater than 1: active if odd, inactive if even
third: if greater than 3: ditto
forth: if greater than 7 ditto

this means that the first output only weights the first two neurons of the previous layer. second output weight the 2nd two. 3rd weights 4 inputs 4-7 (indexing starting at zero). and so on....